In [1]:
import pandas as pd
import numpy as np
import os.path
import recsys as rs
import notipy
import math
import sys
import TopSimilarRecommender as TSR
import ItemBasedRecommender as IBR
import random
from scipy import sparse as sps
import importlib

In [2]:
train = pd.read_csv('Data/train_final.csv','\t')
tracks = pd.read_csv('Data/tracks_final.csv','\t')
pl_info = pd.read_csv('Data/playlists_final.csv','\t')
tgt_playlists = pd.read_csv('Data/target_playlists.csv','\t')
tgt_tracks = pd.read_csv('Data/target_tracks.csv','\t')

In [3]:
train, test, tgt_tracks, tgt_playlists = rs.split_train_test(train, 10, 20, 5, 2517)
tgt_playlists.shape

(4420, 1)

In [4]:
ibr = IBR.ItemBasedRecommender(shrinkage=10,n_el_sim=65)
ibr.fit(tracks,train,tgt_tracks)
print('Model fitted!')

  0%|          | 15/4420 [00:00<00:31, 139.24it/s]

Calculated Indices


  0%|          | 28/100000 [00:00<05:59, 278.29it/s]

(4420, 100000)
Model URM built


100%|██████████| 100000/100000 [21:21<00:00, 78.02it/s]

Similarity built
(100000, 17180)
Model fitted!


In [5]:
tsr= TSR.TopSimilarRecommender(attributes=['artist_id', 'album', 'playcount'],idf=True, n_min_attr=90, n_el_sim=65)
tsr.fit(tracks,tgt_tracks )
print('Model fitted!')

Fixed dataset
Calculated Indices


  0%|          | 71/100000 [00:00<02:20, 709.54it/s]

ICM built
ICM regularized with IDF!


100%|██████████| 100000/100000 [19:53<00:00, 83.79it/s]


Similarity built
Model fitted!


# Recommendation

In [6]:
_, _, IX_tgt_playlists, _ = rs.create_sparse_indexes(playlists=tgt_playlists)
URM = rs.create_tgt_URM(IX_tgt_playlists, tsr.IX_items, train)
URM = URM.tocsr()
print('URM built')

100%|██████████| 4420/4420 [00:12<00:00, 366.67it/s]


URM built


In [9]:
rs = importlib.reload(rs)
recommendetions = np.array([])
for p in IX_tgt_playlists.values:
    avg_sims = URM[p,:].dot(tsr.S).toarray().ravel()
    topTsr = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    avg_sims = URM[p,:].dot(ibr.S).toarray().ravel()
    topIbr = rs.top5_outside_playlist(avg_sims, p, train, IX_tgt_playlists, tsr.IX_tgt_items, False, False)
    top = np.array([])
    if len(topTsr)>2:
        top = np.append(top,(topTsr[0], topTsr[1], topTsr[2], topTsr[3],topTsr[4]))
    else: 
        top = np.append(top,(topTsr[0], topTsr[1], topIbr[0], topIbr[1],topIbr[2]))
    recommendetions = np.append(recommendetions, rs.sub_format(top))
    if (p % 1000 == 0):
        print('Recommended ' + str(p) + ' users over ' + str(IX_tgt_playlists.values.shape[0]))

rec_tsr =  pd.DataFrame({'playlist_id' : IX_tgt_playlists.index.values, 'track_ids' : recommendetions})

Recommended 0 users over 4420


IndexError: index 4 is out of bounds for axis 0 with size 4

# Evaluation

In [ ]:
def evaluate(results, test, eval_metric='MAP'):
    if eval_metric == 'MAP':
        APs = results.apply(calculate_AP, axis=1, args=(test,))
        res = (APs.sum())/results.shape[0]
    return res

def calculate_AP(row, test):
    p_id = row['playlist_id']
    recs = np.fromstring(row['track_ids'], dtype=float, sep=' ')

    AP = 0
    rel_sum = 0
    n_rel_items = min(test[test['playlist_id'] == p_id].shape[0],5)
    for i in range(recs.size):
        rel = 1 if ((test['playlist_id'] == int(p_id)) & (test['track_id'] == recs[i])).any() else 0
        rel_sum += rel
        P = rel_sum/(i+1)
        AP += (P * rel)/n_rel_items

    return AP

In [ ]:
map_eval = evaluate(rec_tsr, test, 'MAP')
print('Evaluation completed!')

In [ ]:
map_eval

# Save to csv

In [48]:
rec_df.to_csv('Submissions/ensemble_submission_notags_alpha07_nel65,shr=2' + str(0) + '.csv', index=False)
print('Results saved as csv!')

Results saved as csv!
